In [1]:
import random
from copy import deepcopy
import heapq

In [2]:
# get the neighbors (valid states) of the current state: also the parent is passed that represents the previous state
def get_neighbors(state, parent=None):
    neighbors = []
    size = len(state)
    zero_x, zero_y = zero_position(state)          # find the position of the 0 tile
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # define the possible moves of the 0 tile
    
    # for each possible move, compute the new position of the 0 tile and if it is valid, create a new state
    for row, col in moves:
        new_row, new_col = zero_x + row, zero_y + col
        if 0 <= new_row < size and 0 <= new_col < size:
            new_state = deepcopy(state)
            new_state[zero_x][zero_y], new_state[new_row][new_col] = new_state[new_row][new_col], new_state[zero_x][zero_y]
            
            # exclude the parent state from the neighbors
            if parent is None or new_state != parent:
                neighbors.append(new_state)
    return neighbors

# initialize the start state by making a number of random moves from the goal state
def init_start_state(goal_state, num_moves):
    state = deepcopy(goal_state)

    # for each move, get the neighbors of the current state and choose one randomly
    for _ in range(num_moves):
        neighbors = get_neighbors(state)
        state = random.choice(neighbors)
    return state

# initialize the goal state given its size
def init_goal_state(n):
    goal_state = [[0] * n for _ in range(n)]
    count = 1
    for i in range(n):
        for j in range(n):
            goal_state[i][j] = count
            count += 1
    goal_state[n - 1][n - 1] = 0
    return goal_state

# compute the distance function combining two heuristics: the Manhattan distance and the number of misplaced tiles
def compute_distance(state, goal):
    distance = 0
    size = len(state)
    misplaced_tiles = 0
    
    for i in range(size):
        for j in range(size):
            value = state[i][j]
            if value != 0:
                goal_x, goal_y = divmod(value - 1, size)
                distance += abs(i - goal_x) + abs(j - goal_y)   # compute the Manhattan distance
                if state[i][j] != goal[i][j]:   # count the number of misplaced tiles
                    misplaced_tiles += 1
    
    return distance + misplaced_tiles

# return the position of the 0 in the current state
def zero_position(state):
    for i in range(len(state)):
        for j in range(len(state[i])):
            if state[i][j] == 0:
                return i, j

# check if the current state is the goal state
def is_goal(state, goal):
    return state == goal

# check if the puzzle is solvable for the following cases:
# - if the size of the puzzle is odd, the puzzle instance is solvable if the number of inversions is even
# - if the size of the puzzle is even, the puzzle instance is solvable if the number of inversions plus the row of the blank tile from the bottom is odd
def is_solvable(state):
    size = len(state)
    list_states = [x for row in state for x in row if x != 0]
    
    inversions = 0
    for i in range(len(list_states)):
        for j in range(i + 1, len(list_states)):
            if list_states[i] > list_states[j]:
                inversions += 1
    
    if size % 2 == 1:
        return inversions % 2 == 0
    
    zero_x, _ = zero_position(state)
    row_from_bottom = size - zero_x
    
    if row_from_bottom % 2 == 0:
        return inversions % 2 == 1
    else:
        return inversions % 2 == 0

# algorithm that solves the puzzle using an implementation of the A* algorithm
def algorithm(start, goal):
    # check if the puzzle is solvable
    if not is_solvable(start):
        print("Puzzle is not solvable.")
        return None, None
    
    # initialize the list of states that need to be evaluated: every element is a tuple with
    # (f, g, current state, previous state)
    list_states = []
    heapq.heappush(list_states, (0, 0, start, None)) # use heap to keep the states sorted by f
    
    # a dictionary that contains the previous state for each state, used at the end to reconstruct the path
    dict_path = {}

    state_g = {str(start): 0}   # the cost of getting from the start state to the current state
    state_f = {str(start): compute_distance(start, goal)} # the total cost of the current state
    
    cost = 0    # the number of states that have been evaluated

    # while there are states to evaluate, pop the state with the lowest f
    while list_states:
        _, g, current, previous = heapq.heappop(list_states)
        if is_goal(current, goal):
            return cost, final_path(dict_path, current)
        
        cost += 1
        
        # for each neighbor of the current state, compute the new state_g and state_f, then add it to the list of states
        # if it is not already present or if the new state_g is better
        for neighbor in get_neighbors(current, previous):
            next_state_g = g + 1    # the cost of getting from the start state to the neighbor
            
            if str(neighbor) not in state_g or next_state_g < state_g[str(neighbor)]:
                state_g[str(neighbor)] = next_state_g
                state_f[str(neighbor)] = next_state_g + compute_distance(neighbor, goal)
                heapq.heappush(list_states, (state_f[str(neighbor)], next_state_g, neighbor, current))
                dict_path[str(neighbor)] = current
    
    return None, None

# construct the path from the start state to the goal state using the dictionary that contains the previous state for each state
def final_path(dict_path, current):
    path = [current]
    while str(current) in dict_path:
        current = dict_path[str(current)]
        path.append(current)
    return path[::-1]


In [ ]:
n = 6                   # size of the puzzle (change it to test different sizes)
moves_factor = 40             # factor that determines the number of random moves to initialize the start state
rand_moves = n * moves_factor     # dynamic number of random permutations of the goal_state to initialize the start state
goal_state = init_goal_state(n)    # initialize the goal_state
start_state = init_start_state(goal_state, rand_moves)  # initialize the start_state

print("Start state:")
for row in start_state:
    print(row)
print()

cost, solution = algorithm(start_state, goal_state)
quality = len(solution) if solution else None
efficiency = quality / cost if quality else None

print("Quality:", quality)
print("Cost:", cost)
print("Efficiency:", efficiency)
print()

if solution:
    print("Solution:")
    for row in solution[-1]:
        print(row)


Start state:
[1, 3, 9, 4, 0, 6]
[8, 27, 2, 16, 5, 12]
[7, 19, 14, 15, 10, 24]
[13, 28, 32, 17, 11, 20]
[25, 26, 21, 23, 18, 29]
[31, 33, 34, 35, 22, 30]

Quality: 67
Cost: 67411
Efficiency: 0.0009939030721988994

Solution:
[1, 2, 3, 4, 5, 6]
[7, 8, 9, 10, 11, 12]
[13, 14, 15, 16, 17, 18]
[19, 20, 21, 22, 23, 24]
[25, 26, 27, 28, 29, 30]
[31, 32, 33, 34, 35, 0]
